In [1]:
# Import required libraries 

import numpy as np
import random
import copy
import time
import cProfile
import magpylib as magpy

In [2]:
class World:
    """
    The World class encapsulates information about the computational domain.
    
    Attributes
    ----------
    <to be written later>
    ----------
    
    """
    
    def __init__(self, ni, nj, nk):
        """
        Initializes the World object with the grid parameters
        ni, nj, nk

        Parameters
        ----------
        ni : int
            number of grid points along x coordinate
        nj : int
            number of grid points along y coordinate
        nk : int
            number of grid points along z coordinate
        ----------
        
        """
        
        self.ni = ni # number of grid points along x coordinate
        self.nj = nj # number of grid points along y coordinate
        self.nk = nk # number of grid points along z coordinate
        
        self.nn = np.zeros(3) # number of grid points along x,y,z coordinates
        
        self.nn[0] = self.ni # number of grid points along x coordinate
        self.nn[1] = self.nj # number of grid points along y coordinate
        self.nn[2] = self.nk # number of grid points along z coordinate
        
        self.x0 = np.zeros(3) # mesh origin vector
        self.dh = np.zeros(3) # mesh spacing vector
        self.xm = np.zeros(3) # mesh max bound vector
        self.xc = np.zeros(3) # mesh centroid
        
        self.EPS_0 = 8.85418782e-12    # epsilon_0
        self.QE    = 1.602176565e-19;  # proton charge
        self.AMU   = 1.660538921e-27   # amu
        self.ME    = 9.10938215e-31;   # mass of electron    
        self.K     = 1.380648e-23;     # Boltzmann's const.
        self.EvToK = self.QE/self.K;   # 1 eV = q/k
        self.MU0   = 4*np.pi*1E-7
        
        self.phi        = np.zeros((self.ni, self.nj, self.nk)) # potential phi
        self.object_id  = np.zeros((self.ni, self.nj, self.nk)) # grid point classifier
        self.phi_new    = np.zeros((self.ni, self.nj, self.nk)) # array for phi_new
        self.R          = np.zeros((self.ni, self.nj, self.nk)) # residual R
        self.R1         = np.zeros((self.ni, self.nj, self.nk)) # residual R
        self.rho        = np.zeros((self.ni, self.nj, self.nk)) # rho
        self.node_vol   = np.zeros((self.ni, self.nj, self.nk)) # node volume
        self.ef         = np.zeros((self.ni, self.nj, self.nk, 3)) # electric field
        self.phi_m      = np.zeros((self.ni, self.nj, self.nk)) # magnetic potential
        self.M          = np.zeros((self.ni, self.nj, self.nk, 3)) # magnetization
        self.H          = np.zeros((self.ni, self.nj, self.nk, 3)) # H field = -grad(phi_m)
        self.B          = np.zeros((self.ni, self.nj, self.nk, 3)) # B field
        self.divM       = np.zeros((self.ni, self.nj, self.nk)) # divergence of M
        self.Rm         = np.zeros((self.ni, self.nj, self.nk)) # residual R for phi_m
        
    def setTime(self, dt, num_ts):
        """
        Set simulation timestep and total number of timesteps.
        
        Parameters
        ----------
        dt : float
            simulation timestep
        num_ts : int
            total number of timesteps
        ----------
        
        """
        self.dt     = dt     # timestep
        self.num_ts = num_ts # total number of timesteps
    
    def setExtents(self, x1, y1, z1, x2, y2, z2):
        
        """
        Set mesh extents and compute grid spacing.

        Parameters
        ----------
        x1 : float
            x-coordinate of world origin
        y1 : float
            y-coordinate of world origin
        z1 : float
            z-coordinate of world origin
        x2 : float
            x-coordinate of world max bound
        y2 : float
            y-coordinate of world max bound
        z2 : float
            z-coordinate of world max bound
        ----------
        
        """
        
        self.x0[0] = x1 # x-coordinate of world origin
        self.x0[1] = y1 # y-coordinate of world origin
        self.x0[2] = z1 # z-coordinate of world origin
        
        self.xm[0] = x2 #x-coordinate of world max bound
        self.xm[1] = y2 #y-coordinate of world max bound
        self.xm[2] = z2 #z-coordinate of world max bound 
        
        # Compute mesh spacing dh, and mesh centroid
        for i in range(3):
            self.dh[i] = (self.xm[i] - self.x0[i]) / (self.nn[i] - 1)
            self.xc[i] = 0.5*(self.x0[i] + self.xm[i]) 
        
        # Compute node volumes
        self.computeNodeVolumes()
        
    def computeNodeVolumes(self): 
        """
        Compute the node volumes.
        Parameters
        ----------
        
        ----------
        
        """
        for i in np.arange(0,self.ni):
            for j in np.arange(0,self.nj):
                for k in np.arange(0,self.nk):
        
                    V = self.dh[0]*self.dh[1]*self.dh[2]
                    if (i==0 or i==self.ni-1): V*=0.5
                    if (j==0 or j==self.nj-1): V*=0.5
                    if (k==0 or k==self.nk-1): V*=0.5
                    
                    self.node_vol[i][j][k] = V
                    
    def pos(self, lc):
        """
        Determine which position from logical coordinate
        
        Parameters
        ----------
        lc : numpy.ndarray
            logical coordinate 
        ----------
        """
        x = np.zeros(3)
        
        x[0] = self.x0[0] + self.dh[0]*lc[0]
        x[1] = self.x0[1] + self.dh[1]*lc[1]
        x[2] = self.x0[2] + self.dh[2]*lc[2]
        
        return x
    
    def inCylinder(self, x):
        """
        Determine if test point lies inside cylinder
        
        Parameters
        ----------
        x_vec : numpy.ndarray
            position vector vector x_vec[0] = (x1[0], x1[1], x1[2])
        ----------
        """
        
        self.p1 = self.cyl_origin - 0.5*self.cyl_axis*self.cyl_height
        self.p2 = self.cyl_origin + 0.5*self.cyl_axis*self.cyl_height
        
        vec = self.p2 - self.p1
        a  = np.dot(x-self.p1, vec)
        b  = np.dot(x-self.p2, vec)
        c  = np.cross(x-self.p1, vec)
        
        if a >=0 and b <=0 and np.linalg.norm(c) <= self.cyl_radius*np.linalg.norm(vec):
            return True
        else:
            return False
        
        
        """
        if x[0]**2+x[1]**2 < self.cyl_radius**2 and x[2]>=p1[2] and x[2]<=p2[2]:
            return True
        else:
            return False
        """
        


    def addCylinder(self, cyl_origin, cyl_axis, cyl_radius, cyl_height, cyl_phi):
        """
        Add cylinder to grid using sugarcube technique.

        Parameters
        ----------
    
        ----------
        
        """
        
        self.cyl_origin = cyl_origin
        self.cyl_axis   = cyl_axis
        self.cyl_radius = cyl_radius
        self.cyl_height = cyl_height
        self.cyl_phi    = cyl_phi
        
        # loop over all i, j, k

        for i in np.arange(0, self.ni):
            for j in np.arange(0, self.nj):
                for k in np.arange(0, self.nk):

                    # check if pos(i,j,k) lies inside sphere
                    x = self.pos(np.array([i,j,k]))

                    if self.inCylinder(x):
                        # if inside, set object_id[i,j,k] = 1,
                        # if inside, set phi[i,j,k] = self.sphere_phi
                        self.object_id[i,j,k] = 1
                        self.phi[i,j,k] = self.cyl_phi
                        
    def addMagneticField(self, current):
        """
        
        """
        
        x1_mm = self.p1[0]*1E3
        y1_mm = self.p1[1]*1E3
        z1_mm = self.p1[2]*1E3
        
        x2_mm = self.p2[0]*1E3
        y2_mm = self.p2[1]*1E3
        z2_mm = self.p2[2]*1E3
        
        s = magpy.source.current.Line(curr=current, vertices=[(x1_mm, y1_mm, z1_mm),(x2_mm, y2_mm, z2_mm)]) 
        
        for i in np.arange(0, self.ni):
            for j in np.arange(0, self.nj):
                for k in np.arange(0, self.nk):

                    # check if pos(i,j,k) lies inside sphere
                    x = self.pos(np.array([i,j,k]))

                    if self.inCylinder(x):
                        # if inside, set phi_m[i,j,k][:] = M
                        self.B[i,j,k,0] = 0.0
                        self.B[i,j,k,1] = 0.0
                        self.B[i,j,k,2] = 0.0

                    else:
                        B = s.getB(x*1E3)*1E-3

                        self.B[i,j,k,0] = B[0]
                        self.B[i,j,k,1] = B[1]
                        self.B[i,j,k,2] = B[2]
                        
              
    def addInlet(self, phi_inlet):
        """
        Add inlet to simulation domain at k = 0.

        Parameters
        ----------
        phi_inlet : float
            inlet potential
        ----------
        """
        self.phi_inlet = phi_inlet
        
        # loop over all i, j at k = 0
        for i in np.arange(0, self.ni):
                for j in np.arange(0, self.nj):
                    # set object_id[i,j,0] = 2
                    # set phi[i,j,0]       = 0
                    self.object_id[i,j,0] = 2
                    self.phi[i,j,0]       = 0
                    
    def markBoundaryCells(self):
        
        for i in np.arange(0, self.ni):
            for j in np.arange(0, self.nj):
                for k in np.arange(0, self.nk):
                    
                    if i==0 or i==self.ni-1 or j==0 or j==self.nj-1 or k==0 or k==self.nk-1:
                        self.object_id[i,j,k]=3
                        
    
    def computeDebyeLength(self, Te, ne):
        """
        Compute the Debye length.

        Parameters
        ----------
        Te : float
            electron temperature, K
        ne : float
            number density, m3
        ----------
        
        """
        
        return np.sqrt(self.EPS_0*self.K*Te/(ne*self.QE**2))
    
    def potentialSolver8(self, max_it, tol):
        """
        Compute the potential field.

        Parameters
        ----------
        max_it : int
            max iterations for Gauss-Seidel
        tol: float
            tolerance for Gauss-Seidel
        ----------
        
        """

        dx2 = 1.0/(self.dh[0]*self.dh[0]); # dx^2
        dy2 = 1.0/(self.dh[1]*self.dh[1]); # dy^2
        dz2 = 1.0/(self.dh[2]*self.dh[2]); # dz^2
    
        L2 = 0.0 # norm
        
        converged = False
        
        # solve potential
        #for it in np.arange(1,max_it+1):
        #    for i in np.arange(0,self.ni):
        #        for j in np.arange(0,self.nj):
        #            for k in np.arange(0,self.nk):
        #if self.object_id[i,j,k] > 0 :
        #    # skip over solid (fixed) nodes
        #    continue
        
        
        #if (i==0):
        #    self.phi[i,j,k] = self.phi[i+1,j,k]
        #elif (i==self.ni-1):
        #    self.phi[i,j,k] = self.phi[i-1,j,k]
        #elif (j==0):
        #    self.phi[i,j,k] = self.phi[i,j+1,k]
        #elif (j==self.nj-1):
        #    self.phi[i,j,k] = self.phi[i,j-1,k]
        #elif (k==0):
        #    self.phi[i,j,k] = self.phi[i,j,k+1]
        #elif (k==self.nk-1):
        #    self.phi[i,j,k] = self.phi[i,j,k-1]
        
        x    = np.where(self.object_id == 0)
        xarr = np.asarray(x)
        u    = xarr.T
        
        for it in np.arange(1,max_it+1):
        
            self.phi[0,:,1:]         = self.phi[1,:,1:]
            self.phi[self.ni-1,:,1:]  = self.phi[self.ni-2,:,1:]

            self.phi[:,0,1:]         = self.phi[:,1,1:]
            self.phi[:,self.nj-1,1:]  = self.phi[:,self.nj-2,1:]

            #self.phi[:,:,0]         = self.phi[:,:,1]
            self.phi[:,:,self.nk-1] = self.phi[:,:,self.nk-2]

            
            #phi_new np.zeros(len(xarr[0,:]))

            self.phi[u[:,0], u[:,1], u[:,2]] = ((self.rho[u[:,0], u[:,1], u[:,2]])/self.EPS_0 +
                        dx2*(self.phi[u[:,0]-1, u[:,1], u[:,2]] + self.phi[u[:,0]+1, u[:,1], u[:,2]]) +
                        dy2*(self.phi[u[:,0], u[:,1]-1, u[:,2]] + self.phi[u[:,0], u[:,1]+1, u[:,2]]) +
                        dz2*(self.phi[u[:,0], u[:,1], u[:,2]-1] + self.phi[u[:,0], u[:,1], u[:,2]+1]))/(2*dx2+2*dy2+2*dz2)

            
                          
            #check for convergence*/
            if it%25==0:
                sum = 0;
                #for i in np.arange(0,self.ni):
                #    for j in np.arange(0,self.nj):
                #        for k in np.arange(0,self.nk):
                #            if self.object_id[i,j,k] > 0 :
                #                # skip over solid (fixed) nodes
                #                continue
                            

                
                self.R[0,:,1:]         = self.phi[0,:,1:] - self.phi[1,:,1:]
                self.R[self.ni-1,:,1:] = self.phi[self.ni-1,:,1:] - self.phi[self.ni-2,:,1:]
                
                self.R[:,0,1:]         = self.phi[:,0,1:] - self.phi[:,1,1:]
                self.R[:,self.nj-1,1:] = self.phi[:,self.nj-1,1:] - self.phi[:,self.nj-2,1:]
                
                #self.R[:,:,0]         = self.phi[:,:,0] - self.phi[:,:,1]
                self.R[:,:,self.nk-1] = self.phi[:,:,self.nk-1] - self.phi[:,:,self.nk-2]

                self.R[u[:,0], u[:,1], u[:,2]] = -self.phi[u[:,0], u[:,1], u[:,2]]*(2*dx2+2*dy2+2*dz2) +\
                (self.rho[u[:,0], u[:,1], u[:,2]])/self.EPS_0 +\
                dx2*(self.phi[u[:,0]-1, u[:,1], u[:,2]] + self.phi[u[:,0]+1, u[:,1], u[:,2]]) +\
                dy2*(self.phi[u[:,0], u[:,1]-1, u[:,2]] + self.phi[u[:,0], u[:,1]+1, u[:,2]]) +\
                dz2*(self.phi[u[:,0], u[:,1], u[:,2]-1] + self.phi[u[:,0], u[:,1], u[:,2]+1])

                sum = np.sum(self.R**2)
            
            
                L2 = np.sqrt(sum/(self.ni*self.nj*self.nk));
                #print("iter: "+str(it)+", L2 = "+str(L2))

                if (L2<tol):
                    converged = True
                    break

        if (converged==False):
            print("Gauss-Seidel failed to converge, L2 = "+str(L2))
        
        return converged
    
    def efSolver2(self):
        """
        Compute the electric field from potential function.

        Parameters
        ----------
        
        ----------
        
        """
        dx = self.dh[0] # dx
        dy = self.dh[1] # dy
        dz = self.dh[2] # dz
        
        """
        for i in np.arange(0, self.ni):
            for j in np.arange(0, self.nj):
                for k in np.arange(0, self.nk):
        """

        ##x-component#
        #if i==0: 
        #x-component#
        """
                    if i==0: 
                        # forward
                        self.ef[i][j][k][0] = -(-3*self.phi[i][j][k]+\
                                               4*self.phi[i+1][j][k]-\
                                               self.phi[i+2][j][k])/(2*dx)
        """
                    
        # forward
        self.ef[0,0:self.nj,0:self.nk,0] = -(-3*self.phi[0,0:self.nj,0:self.nk]+\
                               4*self.phi[1,0:self.nj,0:self.nk]-\
                               self.phi[2,0:self.nj,0:self.nk])/(2*dx)
        
        #elif i==self.ni-1:  
        """
        elif i==self.ni-1:  
                        # backward
                        self.ef[i][j][k][0] = -(self.phi[i-2][j][k]-\
                                               4*self.phi[i-1][j][k]+\
                                               3*self.phi[i][j][k])/(2*dx)
        """           
        # backward
        self.ef[self.ni-1,0:self.nj,0:self.nk,0] = -(self.phi[self.ni-3,0:self.nj,0:self.nk]-\
                                   4*self.phi[self.ni-2,0:self.nj,0:self.nk]+\
                                   3*self.phi[self.ni-1,0:self.nj,0:self.nk])/(2*dx)
        """
        else: 
            #central
            self.ef[i][j][k][0] = -(self.phi[i+1][j][k] - \
                                    self.phi[i-1][j][k])/(2*dx)
        """ 
        #central
        self.ef[1:self.ni-1,0:self.nj,0:self.nk,0] = -(self.phi[2:self.ni,0:self.nj,0:self.nk] - \
                                self.phi[0:self.ni-2,0:self.nj,0:self.nk])/(2*dx)


        #y-component
        #if j==0:
        """
        if j==0:
                        self.ef[i][j][k][1] = -(-3*self.phi[i][j][k] + \
                                                4*self.phi[i][j+1][k]-\
                                                self.phi[i][j+2][k])/(2*dy)
                    
        """
        self.ef[0:self.ni,0,0:self.nk,1] = -(-3*self.phi[0:self.ni,0,0:self.nk] + \
                                    4*self.phi[0:self.ni,1,0:self.nk]-\
                                    self.phi[0:self.ni,2,0:self.nk])/(2*dy)
        #elif j==self.nj-1:
        """
        elif j==self.nj-1:
                        self.ef[i][j][k][1] = -(self.phi[i][j-2][k] - \
                                                4*self.phi[i][j-1][k] +\
                                                3*self.phi[i][j][k])/(2*dy)
                    
        """
        self.ef[0:self.ni,self.nj-1,0:self.nk,1] = -(self.phi[0:self.ni,self.nj-3,0:self.nk] - \
                                    4*self.phi[0:self.ni,self.nj-2,0:self.nk] +\
                                    3*self.phi[0:self.ni,self.nj-1,0:self.nk])/(2*dy)
        #else:
        """
        else:
                         self.ef[i][j][k][1] = -(self.phi[i][j+1][k] - \
                                                 self.phi[i][j-1][k])/(2*dy)

        """
        self.ef[0:self.ni,1:self.nj-1,0:self.nk,1] = -(self.phi[0:self.ni,2:self.nj,0:self.nk] - \
                                     self.phi[0:self.ni,0:self.nj-2,0:self.nk])/(2*dy)

        #z-component
        '''
        if k==0:
            self.ef[i][j][k][2] = -(-3*self.phi[i][j][k] + \
                                    4*self.phi[i][j][k+1]-
                                    self.phi[i][j][k+2])/(2*dz)
            
        '''
        #z-component
        #if k==0:
        self.ef[0:self.ni,0:self.nj,0,2] = -(-3*self.phi[0:self.ni,0:self.nj,0] + \
                                4*self.phi[0:self.ni,0:self.nj,1]-
                                self.phi[0:self.ni,0:self.nj,2])/(2*dz)

        """
        elif k==self.nk-1:
            self.ef[i][j][k][2] = -(self.phi[i][j][k-2] - \
                                    4*self.phi[i][j][k-1]    + \
                                    3*self.phi[i][j][k])/(2*dz)
        """
        
        #elif k==self.nk-1:
        self.ef[0:self.ni,0:self.nj,self.nk-1,2] = -(self.phi[0:self.ni,0:self.nj,self.nk-3] - \
                                    4*self.phi[0:self.ni,0:self.nj,self.nk-2]    + \
                                    3*self.phi[0:self.ni,0:self.nj,self.nk-1])/(2*dz) 
        """
        else:
            self.ef[i][j][k][2] = -(self.phi[i][j][k+1] - \
                                    self.phi[i][j][k-1])/(2*dz)
        """
        #else:
        self.ef[0:self.ni,0:self.nj,1:self.nk-1,2] = -(self.phi[0:self.ni,0:self.nj,2:self.nk] - \
                                    self.phi[0:self.ni,0:self.nj,0:self.nk-2])/(2*dz)
    
    
    def XtoL(self, x):
        """
        Determine which cell a particle at position vector 
        x belongs to.
        
        Parameters
        ----------
        x : numpy.ndarray
            position vector 
        ----------
        """
        lc = np.zeros(3)
        
        lc[0] = (x[0]-self.x0[0])/self.dh[0];
        lc[1] = (x[1]-self.x0[1])/self.dh[1];
        lc[2] = (x[2]-self.x0[2])/self.dh[2];
        
        return lc
    
    
    def inBounds(self, x):
        """
        Determine if position x is within mesh bounds
        
        Parameters
        ----------
        x_vec : numpy.ndarray
            position vector vector x_vec[0] = (x1[0], x1[1], x1[2])
        ----------
        """
        
        for i in range(0,3):
            if (x[i] < self.x0[i] or x[i] >= self.xm[i]):
                return False
        
        return True
        
    def XtoLvec(self, x_vec):
        """
        Determine which cell a particle at position vector 
        x belongs to.
        
        Parameters
        ----------
        x_vec : numpy.ndarray
            position vector vector x_vec[0] = (x1[0], x1[1], x1[2])
        ----------
        """
        lc = np.zeros((len(x_vec[:,0]),3))
        
        lc[:,0] = (x_vec[:,0]-self.x0[0])/self.dh[0];
        lc[:,1] = (x_vec[:,1]-self.x0[1])/self.dh[1];
        lc[:,2] = (x_vec[:,2]-self.x0[2])/self.dh[2];
        
        return lc
    
    def addSpeciesList(self, speciesList):
        self.speciesList = speciesList
    
    def computeChargeDensity(self):
        """
        Compute the charge density.
        
        Parameters
        ----------
     
        ----------
        """
        
        self.rho = np.zeros((self.ni, self.nj, self.nk))
        
        for species in self.speciesList:
            if species.charge!=0:
                self.rho += species.charge*species.den         
    
    

In [3]:
class Particle:
    """
    The Particle class encapsulates information about the particles 
    used in the simulation.
    
    Attributes
    ----------
    
    ----------
    
    """
    
    def __init__(self, pos, vel, mpw):
        """
        Initializes the Particle object with the position, 
        speed, and macrparticle weight.

        Parameters
        ----------
        pos : numpy.ndarray
            particle position vector
        vel : numpy.ndarray
            particle velocity vector
        mpw : float
            macroparticle weight  
        ----------
        
        """
        
        self.pos = pos
        self.vel = vel
        self.mpw = mpw
        
    def updateVelocityBoris(self, dt, E, B):
        
        v_minus = self.vel + (self.q/self.m)*E*dt/2
        t       = (self.q/self.m)*B*dt/2
        v_prime = v_minus + np.cross(v_minus, t)
        s       = 2.0*t/(1+np.dot(t,t))
        v_plus  = v_minus + np.cross(v_prime, s)
        self.vel= v_plus + (self.q/self.m)*E*dt/2      
        


class Species:
    """
    The Species class encapsulates information about the species 
    used in the simulation.
    
    Attributes
    ----------
    
    ----------
    
    """
    def __init__(self, name, mass, charge, mpw0, worldObj):
        """
        Initializes the Species object with the name, mass,
        charge.

        Parameters
        ----------
        name : str
            species name
        mass : float
            species mass
        charge : float
            species charge
        mpw : float
            macroparticle weight  
        ----------

        """
        
        self.particleList = []
        
        self.name   = name
        self.mass   = mass
        self.charge = charge
        self.mpw0   = mpw0
        
        self.den    = np.zeros((worldObj.ni, worldObj.nj, worldObj.nk))
        
        self.worldObj = worldObj

    def ColdBeamSource(self, v_drift, den):
        """
        Cold beam source

        Parameters
        ----------
        v_drift : float
            drift speed
        den : float
            number density
        ----------
        
        """
        
        Lx = self.worldObj.dh[0]*(self.worldObj.ni-1)
        Ly = self.worldObj.dh[1]*(self.worldObj.nj-1)
        
        A   = Lx*Ly
        
        num_real = den*v_drift*A*self.worldObj.dt
        
        num_sim  = int(num_real / self.mpw0 + random.random())
        
        for i in range(0, num_sim):
            pos = np.zeros(3)
            vel = np.zeros(3)
            
            pos[0] = self.worldObj.x0[0] + random.random()*Lx
            pos[1] = self.worldObj.x0[1] + random.random()*Ly
            pos[2] = self.worldObj.x0[2]
            
            vel[0] = 0.0
            vel[1] = 0.0
            vel[2] = v_drift
            
            self.addParticle(pos,vel,self.mpw0)     
    
    def addParticle(self, pos, vel, mpw):
        """random.random()
        add a particle to particleList

         Parameters
        ----------
        pos : numpy.ndarray
            particle position vector
        vel : numpy.ndarray
            particle velocity vector
        mpw : float
            macroparticle weight  
        ----------
        
        """
        #get logical coordinate of particle's position
        lc = self.worldObj.XtoL(pos)

        #electric field at particle position
        ef_part = self.gather_ef(lc)
        bf_part = self.gather_bf(lc)

        #rewind velocity back by 0.5*dt*ef
        v_minus = vel + (self.charge/self.mass)*ef_part*(-0.5*self.worldObj.dt)/2
        t       = (self.charge/self.mass)*bf_part*(-0.5*self.worldObj.dt)/2
        v_prime = v_minus + np.cross(v_minus, t)
        s       = 2.0*t/(1+np.dot(t,t))
        v_plus  = v_minus + np.cross(v_prime, s)
        vel     = v_plus + (self.charge/self.mass)*ef_part*(-0.5*self.worldObj.dt)/2      
        
        #add particle to list
        self.particleList.append(Particle(pos, vel, mpw))
        
        
    def loadParticlesBoxQS(self, x1, x2, num_den, num_mp):
        """
        loads randomly distributed particles in a x1-x2 box 
        representing num_den number density

        Parameters
        ----------
        x1 : numpy.ndarray
            origin of bounding box
        x2 : numpy.ndarray
            max. bound corner of box
        num_den : float
            number density
        num_mp  : numpy.ndarray
        ----------

        """
        box_vol  = (x2[0]-x1[0])*(x2[1]-x1[1])*(x2[2]-x1[2]) # box vol.
        num_real = num_den * box_vol;                   #number of real particles
        num_mp_tot = (num_mp[0]-1)*(num_mp[1]-1)*(num_mp[2]-1)
        mpw      = num_real/num_mp_tot;                     # macroparticle weight
        
        self.box_vol = box_vol
        self.num_real= num_real
        self.mpw     = mpw
        
        di = (x2[0]-x1[0])/(num_mp[0]-1);
        dj = (x2[1]-x1[1])/(num_mp[1]-1);
        dk = (x2[2]-x1[2])/(num_mp[2]-1);

        #load particles on a equally spaced grid
        
        for i in np.arange(0,num_mp[0]):
            for j in np.arange(0,num_mp[1]):
                for k in np.arange(0,num_mp[2]):
                    
                    # sample random position
                    pos = np.zeros(3)
                    vel = np.zeros(3)
                    
                    pos[0] = x1[0] + i*di;
                    pos[1] = x1[1] + j*dj;
                    pos[2] = x1[2] + k*dk;

                    # shift particles on max faces back to the domain
                    if (pos[0]==x2[0]): pos[0]-=1e-4*di;
                    if (pos[1]==x2[1]): pos[1]-=1e-4*dj;
                    if (pos[2]==x2[2]): pos[2]-=1e-4*dk;

                    w = 1;
                    if (i==0 or i==num_mp[0]-1): w*=0.5
                    if (j==0 or j==num_mp[1]-1): w*=0.5
                    if (k==0 or k==num_mp[2]-1): w*=0.5

                    #set initial velocity
                    vel[0] = 0.0
                    vel[1] = 0.0
                    vel[2] = 0.0

                    self.addParticle(pos,vel,mpw*w)
    
    
    def scatter_den(self, lc, value):
        """
        scatters scalar value onto a field at logical coordinate lc

        Parameters
        ----------
        lc : numpy.ndarray
            logical coordinate 
        ----------
        """
        
        if lc [0] < 0 or \
             lc [0] >=self.worldObj.ni-1 or \
             lc [1] < 0 or \
             lc [1] >=self.worldObj.nj-1 or \
             lc [2] < 0 or \
             lc [2] >=self.worldObj.nk-1:
            
            print("WARNING: point outside domain")
             
        i  = int(lc[0])
        di = lc[0]-i

        j  = int(lc[1])
        dj = lc[1]-j

        k  = int(lc[2])
        dk = lc[2]-k

        self.den[i][j][k]      += value*(1-di)*(1-dj)*(1-dk)
        self.den[i+1][j][k]    += value*(di)*(1-dj)*(1-dk)
        self.den[i+1][j+1][k]  += value*(di)*(dj)*(1-dk)
        self.den[i][j+1][k]    += value*(1-di)*(dj)*(1-dk)
        self.den[i][j][k+1]    += value*(1-di)*(1-dj)*(dk)
        self.den[i+1][j][k+1]  += value*(di)*(1-dj)*(dk)
        self.den[i+1][j+1][k+1]+= value*(di)*(dj)*(dk)
        self.den[i][j+1][k+1]  += value*(1-di)*(dj)*(dk)
    
    
    def computeNumberDensity(self):
        """
        Compute particle number density

        Parameters
        ----------
        
        ----------
        """
        self.den = np.zeros((self.worldObj.ni, self.worldObj.nj, self.worldObj.nk))
        
        for particle in self.particleList:
            lc = self.worldObj.XtoL(particle.pos)
            self.scatter_den(lc, self.mpw0)
            
        self.den = self.den / self.worldObj.node_vol
    
    
    def gather_ef(self, lc):
        """
        gathers field value at logical coordinate lc

        Parameters
        ----------
        lc : numpy.ndarray
            logical coordinate 
        data : numpy.ndarray
            electric field array
        ----------
        """
        
        if lc [0] < 0 or \
             lc [0] >=self.worldObj.ni-1 or \
             lc [1] < 0 or \
             lc [1] >=self.worldObj.nj-1 or \
             lc [2] < 0 or \
             lc [2] >=self.worldObj.nk-1:
            
            print("WARNING: point outside domain")
             
        i  = int(lc[0])
        di = lc[0]-i

        j  = int(lc[1])
        dj = lc[1]-j

        k  = int(lc[2])
        dk = lc[2]-k

        # gather electric field onto particle position
         
        ef_x  = self.worldObj.ef[i][j][k][0]*(1-di)*(1-dj)*(1-dk)+\
                self.worldObj.ef[i+1][j][k][0]*(di)*(1-dj)*(1-dk)+\
                self.worldObj.ef[i+1][j+1][k][0]*(di)*(dj)*(1-dk)+\
                self.worldObj.ef[i][j+1][k][0]*(1-di)*(dj)*(1-dk)+\
                self.worldObj.ef[i][j][k+1][0]*(1-di)*(1-dj)*(dk)+\
                self.worldObj.ef[i+1][j][k+1][0]*(di)*(1-dj)*(dk)+\
                self.worldObj.ef[i+1][j+1][k+1][0]*(di)*(dj)*(dk)+\
                self.worldObj.ef[i][j+1][k+1][0]*(1-di)*(dj)*(dk)
        
        ef_y  = self.worldObj.ef[i][j][k][1]*(1-di)*(1-dj)*(1-dk)+\
                self.worldObj.ef[i+1][j][k][1]*(di)*(1-dj)*(1-dk)+\
                self.worldObj.ef[i+1][j+1][k][1]*(di)*(dj)*(1-dk)+\
                self.worldObj.ef[i][j+1][k][1]*(1-di)*(dj)*(1-dk)+\
                self.worldObj.ef[i][j][k+1][1]*(1-di)*(1-dj)*(dk)+\
                self.worldObj.ef[i+1][j][k+1][1]*(di)*(1-dj)*(dk)+\
                self.worldObj.ef[i+1][j+1][k+1][1]*(di)*(dj)*(dk)+\
                self.worldObj.ef[i][j+1][k+1][1]*(1-di)*(dj)*(dk)
        
        ef_z  = self.worldObj.ef[i][j][k][2]*(1-di)*(1-dj)*(1-dk)+\
                self.worldObj.ef[i+1][j][k][2]*(di)*(1-dj)*(1-dk)+\
                self.worldObj.ef[i+1][j+1][k][2]*(di)*(dj)*(1-dk)+\
                self.worldObj.ef[i][j+1][k][2]*(1-di)*(dj)*(1-dk)+\
                self.worldObj.ef[i][j][k+1][2]*(1-di)*(1-dj)*(dk)+\
                self.worldObj.ef[i+1][j][k+1][2]*(di)*(1-dj)*(dk)+\
                self.worldObj.ef[i+1][j+1][k+1][2]*(di)*(dj)*(dk)+\
                self.worldObj.ef[i][j+1][k+1][2]*(1-di)*(dj)*(dk)
        
        ef_part = np.array([ef_x, ef_y, ef_z])
        
        return ef_part
    
    def gather_bf(self, lc):
        """
        gathers field value at logical coordinate lc

        Parameters
        ----------
        lc : numpy.ndarray
            logical coordinate 
        data : numpy.ndarray
            electric field array
        ----------
        """
        
        if lc [0] < 0 or \
             lc [0] >=self.worldObj.ni-1 or \
             lc [1] < 0 or \
             lc [1] >=self.worldObj.nj-1 or \
             lc [2] < 0 or \
             lc [2] >=self.worldObj.nk-1:
            
            print("WARNING: point outside domain")
             
        i  = int(lc[0])
        di = lc[0]-i

        j  = int(lc[1])
        dj = lc[1]-j

        k  = int(lc[2])
        dk = lc[2]-k

        # gather electric field onto particle position
         
        B_x  = self.worldObj.B[i][j][k][0]*(1-di)*(1-dj)*(1-dk)+\
                self.worldObj.B[i+1][j][k][0]*(di)*(1-dj)*(1-dk)+\
                self.worldObj.B[i+1][j+1][k][0]*(di)*(dj)*(1-dk)+\
                self.worldObj.B[i][j+1][k][0]*(1-di)*(dj)*(1-dk)+\
                self.worldObj.B[i][j][k+1][0]*(1-di)*(1-dj)*(dk)+\
                self.worldObj.B[i+1][j][k+1][0]*(di)*(1-dj)*(dk)+\
                self.worldObj.B[i+1][j+1][k+1][0]*(di)*(dj)*(dk)+\
                self.worldObj.B[i][j+1][k+1][0]*(1-di)*(dj)*(dk)
        
        B_y  = self.worldObj.B[i][j][k][1]*(1-di)*(1-dj)*(1-dk)+\
                self.worldObj.B[i+1][j][k][1]*(di)*(1-dj)*(1-dk)+\
                self.worldObj.B[i+1][j+1][k][1]*(di)*(dj)*(1-dk)+\
                self.worldObj.B[i][j+1][k][1]*(1-di)*(dj)*(1-dk)+\
                self.worldObj.B[i][j][k+1][1]*(1-di)*(1-dj)*(dk)+\
                self.worldObj.B[i+1][j][k+1][1]*(di)*(1-dj)*(dk)+\
                self.worldObj.B[i+1][j+1][k+1][1]*(di)*(dj)*(dk)+\
                self.worldObj.B[i][j+1][k+1][1]*(1-di)*(dj)*(dk)
        
        B_z  = self.worldObj.B[i][j][k][2]*(1-di)*(1-dj)*(1-dk)+\
                self.worldObj.B[i+1][j][k][2]*(di)*(1-dj)*(1-dk)+\
                self.worldObj.B[i+1][j+1][k][2]*(di)*(dj)*(1-dk)+\
                self.worldObj.B[i][j+1][k][2]*(1-di)*(dj)*(1-dk)+\
                self.worldObj.B[i][j][k+1][2]*(1-di)*(1-dj)*(dk)+\
                self.worldObj.B[i+1][j][k+1][2]*(di)*(1-dj)*(dk)+\
                self.worldObj.B[i+1][j+1][k+1][2]*(di)*(dj)*(dk)+\
                self.worldObj.B[i][j+1][k+1][2]*(1-di)*(dj)*(dk)
        
        B_part = np.array([B_x, B_y, B_z])
        
        return B_part
    
    def advance(self):
        #get the time step
        dt = self.worldObj.dt

        #save mesh bounds
        x0 = self.worldObj.x0
        xm = self.worldObj.xm

        # loop over all particles
        for particle in self.particleList:
        
            #get logical coordinate of particle's position
            lc = self.worldObj.XtoL(particle.pos)

            #electric field at particle position
            ef_part = self.gather_ef(lc)
            bf_part = self.gather_bf(lc)
            
            
            #update velocity from F=qE
            
            v_minus = particle.vel + (self.charge/self.mass)*ef_part*(self.worldObj.dt)/2
            t       = (self.charge/self.mass)*bf_part*(self.worldObj.dt)/2
            v_prime = v_minus + np.cross(v_minus, t)
            s       = 2.0*t/(1+np.dot(t,t))
            v_plus  = v_minus + np.cross(v_prime, s)
            vel     = v_plus + (self.charge/self.mass)*ef_part*(self.worldObj.dt)/2 
            
            particle.vel = vel

            #update position from v=dx/dt
            particle.pos += particle.vel*dt

            if self.worldObj.inCylinder(particle.pos) or self.worldObj.inBounds(particle.pos)==False:
                particle.mpw = 0.0
                
        npp = len(self.particleList)

        p = 0

        while p < npp:
                if self.particleList[p].mpw == 0:
                    del self.particleList[p]

                    npp = npp - 1
                    p  = p - 1

                p = p+1

In [9]:
world=World(21,21,41)
world.setTime(1E-7,10)
world.setExtents(-0.1, -0.1, 0.0 , 0.1, 0.1, 0.4)
world.addCylinder(np.array([0.0,0.0,0.2]), np.array([0,0,1]), 0.02, 0.2, -100.0)

In [10]:
world.addInlet(0.0)
world.markBoundaryCells()
world.addMagneticField(100)

In [11]:
world.potentialSolver8(20000, 1E-4)

True

In [12]:
species1 = Species("O++", 16*world.AMU, 2*world.QE, 1E2, world)
world.addSpeciesList([species1])

species1.computeNumberDensity()
world.computeChargeDensity()

world.potentialSolver8(30000, 1E-4)
world.efSolver2()

In [13]:
for i in range(1,401):
    
    print("ts = "+str(i)+", nO+: "+str(len(species1.particleList)))
    
    species1.ColdBeamSource(8000.0, 1E10)
    species1.advance()
    species1.computeNumberDensity()
    
    world.computeChargeDensity()
    
    world.potentialSolver8(30000, 1E-4)
    
    world.efSolver2()
    
    if i==1 or i%40==0:

        x_arr1 = np.zeros(len(species1.particleList))
        y_arr1 = np.zeros(len(species1.particleList))
        z_arr1 = np.zeros(len(species1.particleList))
        
        vx_arr1 = np.zeros(len(species1.particleList))
        vy_arr1 = np.zeros(len(species1.particleList))
        vz_arr1 = np.zeros(len(species1.particleList))

        for j in np.arange(0,len(species1.particleList)):
            pos = species1.particleList[j].pos
            x_arr1[j] = pos[0]
            y_arr1[j] = pos[1]
            z_arr1[j] = pos[2]

            vel = species1.particleList[j].vel
            vx_arr1[j] = vel[0]
            vy_arr1[j] = vel[1]
            vz_arr1[j] = vel[2]

        
        np.savetxt('01species_1_'+str(i).zfill(4)+'_x'+'.txt', x_arr1, fmt='%1.4f')
        np.savetxt('01species_1_'+str(i).zfill(4)+'_y'+'.txt', y_arr1, fmt='%1.4f')
        np.savetxt('01species_1_'+str(i).zfill(4)+'_z'+'.txt', z_arr1, fmt='%1.4f')
        
        np.savetxt('01species_1_'+str(i).zfill(4)+'_vx'+'.txt', vx_arr1, fmt='%1.4f')
        np.savetxt('01species_1_'+str(i).zfill(4)+'_vy'+'.txt', vy_arr1, fmt='%1.4f')
        np.savetxt('01species_1_'+str(i).zfill(4)+'_vz'+'.txt', vz_arr1, fmt='%1.4f')

        np.save('01phi_'+str(i).zfill(4)+'.npy', world.phi)
        np.save('01rho_'+str(i).zfill(4)+'.npy', world.rho)
        np.save('01den_'+str(i).zfill(4)+'.npy', species1.den)
    

ts = 1, nO+: 0
ts = 2, nO+: 3200
ts = 3, nO+: 6400
ts = 4, nO+: 9600
ts = 5, nO+: 12800
ts = 6, nO+: 16000
ts = 7, nO+: 19200
ts = 8, nO+: 22400
ts = 9, nO+: 25600
ts = 10, nO+: 28800
ts = 11, nO+: 32000
ts = 12, nO+: 35200
ts = 13, nO+: 38400
ts = 14, nO+: 41599
ts = 15, nO+: 44799
ts = 16, nO+: 47999
ts = 17, nO+: 51199
ts = 18, nO+: 54399
ts = 19, nO+: 57599
ts = 20, nO+: 60798
ts = 21, nO+: 63997
ts = 22, nO+: 67196
ts = 23, nO+: 70394
ts = 24, nO+: 73592
ts = 25, nO+: 76792
ts = 26, nO+: 79991
ts = 27, nO+: 83189
ts = 28, nO+: 86386
ts = 29, nO+: 89585
ts = 30, nO+: 92781
ts = 31, nO+: 95978
ts = 32, nO+: 99176
ts = 33, nO+: 102371
ts = 34, nO+: 105565
ts = 35, nO+: 108759
ts = 36, nO+: 111953
ts = 37, nO+: 115148
ts = 38, nO+: 118337
ts = 39, nO+: 121532
ts = 40, nO+: 124724
ts = 41, nO+: 127913
ts = 42, nO+: 130935
ts = 43, nO+: 133985
ts = 44, nO+: 137030
ts = 45, nO+: 140039
ts = 46, nO+: 143026
ts = 47, nO+: 146014
ts = 48, nO+: 148970
ts = 49, nO+: 151906
ts = 50, nO+: 15480

KeyboardInterrupt: 